In [1]:
import json
import os

def scale_bboxes(coco_json_path, output_json_path, scale_factor=2):
    # JSON 파일 불러오기
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)
    
    # images의 file_name을 변경
    for img in coco_data['images']:
        file_name = img['file_name']
        new_file_name = img['file_name'][6:10] + '_x2_SR.png'
        img['file_name'] = new_file_name

    # annotations의 bbox 크기를 조정
    for annotation in coco_data['annotations']:
        x, y, width, height = annotation['bbox']
        area = annotation['area']
        
        # bbox 크기를 2배로 늘리기
        new_x = x * scale_factor
        new_y = y * scale_factor
        new_width = width * scale_factor
        new_height = height * scale_factor
        new_area = area * scale_factor * scale_factor
        
        # 새로운 bbox 저장
        annotation['bbox'] = [new_x, new_y, new_width, new_height]
        annotation['area'] = new_area
    
    # 새로운 JSON 파일로 저장
    with open(output_json_path, 'w') as f:
        json.dump(coco_data, f, indent=4)

# 경로 설정
input_json = '../../../dataset/train_0.json'  # 기존 COCO json 파일 경로
output_json = './experiment/test/train_0_sr_x2.json'  # 수정된 json 파일을 저장할 경로

# bbox 크기 2배로 늘리고 저장
scale_bboxes(input_json, output_json)

In [1]:
import json

def assign_unique_annotation_ids(coco_json_path, updated_json_path):
    """
    COCO 데이터 형식의 annotation JSON 파일에서 고유한 annotation ID를 부여하는 함수
    :param coco_json_path: 원본 COCO 형식의 annotation 파일 경로
    :param updated_json_path: 업데이트된 COCO 형식의 annotation 파일 저장 경로
    """
    
    # JSON 파일 불러오기
    with open(coco_json_path, 'r') as file:
        coco_data = json.load(file)

    # 현재 annotation에서 가장 큰 ID 찾기
    existing_ids = [ann['id'] for ann in coco_data['annotations']]
    max_id = max(existing_ids) if existing_ids else 0

    # annotation에 대해 고유한 ID 부여
    for idx, annotation in enumerate(coco_data['annotations']):
        # 새로운 고유 ID 생성
        annotation['id'] = max_id + idx + 1  # 고유 ID 부여 (기존 최대 ID + 1부터 시작)

    # 업데이트된 데이터를 새로운 JSON 파일로 저장
    with open(updated_json_path, 'w') as file:
        json.dump(coco_data, file, indent=2)

    print(f"Updated annotations saved to {updated_json_path}")

# 함수 호출 예시
coco_json_path = '/data/ephemeral/home/sr_dataset/train_0_x2_SR_center.json'  # 원본 COCO 형식 JSON 파일 경로
updated_json_path = '/data/ephemeral/home/sr_dataset/updated_train_0_x2_SR_center.json'  # 업데이트된 파일 경로

assign_unique_annotation_ids(coco_json_path, updated_json_path)


Updated annotations saved to /data/ephemeral/home/sr_dataset/updated_train_0_x2_SR_center.json


In [3]:
import json

def merge_coco_json_files(json_path_1, json_path_2, merged_json_path):
    """
    두 개의 COCO 형식의 annotation 파일을 하나로 합치는 함수
    :param json_path_1: 첫 번째 COCO JSON 파일 경로
    :param json_path_2: 두 번째 COCO JSON 파일 경로
    :param merged_json_path: 병합된 COCO JSON 파일 저장 경로
    """
    # 두 개의 JSON 파일 불러오기
    with open(json_path_1, 'r') as file1, open(json_path_2, 'r') as file2:
        coco_data_1 = json.load(file1)
        coco_data_2 = json.load(file2)

    # 병합할 데이터 초기화
    merged_data = {
        'images': [],
        'annotations': [],
        'categories': coco_data_1['categories']  # 카테고리 동일
    }

    # 카테고리 ID는 동일한 것으로 가정하고 진행
    assert coco_data_1['categories'] == coco_data_2['categories'], "카테고리가 동일해야 합니다."

    # 이미지와 어노테이션 ID들의 최대값을 계산하여 중복 방지
    max_image_id = max([img['id'] for img in coco_data_1['images']] + [0])
    max_annotation_id = max([ann['id'] for ann in coco_data_1['annotations']] + [0])

    # 첫 번째 JSON 파일 데이터 추가
    merged_data['images'].extend(coco_data_1['images'])
    merged_data['annotations'].extend(coco_data_1['annotations'])

    # 두 번째 JSON 파일 데이터 추가 (ID 중복 방지를 위해 ID 업데이트)
    for image in coco_data_2['images']:
        new_image = image.copy()
        new_image['id'] += max_image_id  # 이미지 ID 업데이트
        new_image['file_name'] = 'centerimages/' + image['file_name']
        merged_data['images'].append(new_image)

    for annotation in coco_data_2['annotations']:
        new_annotation = annotation.copy()
        new_annotation['id'] += max_annotation_id  # 어노테이션 ID 업데이트
        new_annotation['image_id'] += max_image_id  # 이미지 ID도 같이 업데이트
        merged_data['annotations'].append(new_annotation)

    # 병합된 JSON 파일 저장
    with open(merged_json_path, 'w') as merged_file:
        json.dump(merged_data, merged_file, indent=2)

    print(f"병합된 파일이 {merged_json_path}에 저장되었습니다.")

# 함수 사용 예시
json_path_1 = '/data/ephemeral/home/dataset/train_0.json'  # 첫 번째 COCO JSON 파일 경로
json_path_2 = '/data/ephemeral/home/dataset/updated_train_0_x2_SR_center.json'  # 두 번째 COCO JSON 파일 경로
merged_json_path = '/data/ephemeral/home/dataset/merged_train_0_x2_SR_center.json'  # 병합된 COCO JSON 파일 경로

merge_coco_json_files(json_path_1, json_path_2, merged_json_path)


병합된 파일이 /data/ephemeral/home/dataset/merged_train_0_x2_SR_center.json에 저장되었습니다.
